In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import time
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
prot = md.load_pdb('/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/6q5i_GLU_cv.pdb')
topology = prot.topology

g1 = [atom.index for atom in topology.chain(0).atoms]
g2 = [atom.index for atom in topology.chain(1).atoms]
g3 = [atom.index for atom in topology.chain(2).atoms]
g4 = [atom.index for atom in topology.chain(3).atoms]

In [ ]:
coord = md.load('apart-lastframe.inpcrd', top = 'apart-lastframe.prmtop')

In [ ]:
phi_array = md.compute_phi(coord)
psi_array = md.compute_psi(coord)

phi_atoms = phi_array[0].tolist()
psi_atoms = psi_array[0].tolist()

phi_atoms_res_5_25 = phi_atoms[4:25]+phi_atoms[34:55]+phi_atoms[64:85]+phi_atoms[94:115]
psi_atoms_res_5_25 = psi_atoms[4:25]+psi_atoms[34:55]+psi_atoms[64:85]+psi_atoms[94:115]

In [ ]:
start_time = time.time()

In [ ]:
prmtop = AmberPrmtopFile('apart-lastframe.prmtop')
inpcrd = AmberInpcrdFile('apart-lastframe.inpcrd')
pdb = PDBFile('/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/6q5i_GLU_cv.pdb')

system = prmtop.createSystem(nonbondedMethod=NoCutoff, constraints=HBonds, hydrogenMass=1.5*amu, 
                             implicitSolvent=OBC2)

expr = 'distance(g1,g2)+distance(g1,g3)+distance(g1,g4)+distance(g2,g3)+distance(g2,g4)+distance(g3,g4)'
#in metadynamics, the expression inside the Force object needs to be the CV, not an energy expression

dist_force_1 = openmm.CustomCentroidBondForce(4, expr)
dist_force_1.addGroup(g1)
dist_force_1.addGroup(g2)
dist_force_1.addGroup(g3)
dist_force_1.addGroup(g4)
dist_force_1.addBond([0,1,2,3])
#dist_force.setForceGroup(1)
#do not add force to the system manually, it will get added in the metadynamics step as a CV force
dist_bias_1 = BiasVariable(dist_force_1, 0.1, 100, 0.5, False)

#average alpha helix phi and psi dihedrals
phi_avg = (-57.0*np.pi)/180.0
psi_avg = (-47.0*np.pi)/180.0

#Ramachandran plot region for right-handed alpha-helix (in deg)
phi_min = -130.0
phi_max = -30.0
psi_min = -68.0
psi_max = 30.0

phi0 = (phi_min +phi_max)/2.0
psi0 = (psi_min+psi_max)/2.0
phicutoff = abs(phi_max-phi_min)/2.0
psicutoff = abs(psi_max-psi_min)/2.0
        
phiforce = CustomTorsionForce('select({},{},{})'.format('step(-k_dihed*cos(theta-phi0)-(-k_dihed*cos_phicutoff))', '-k_dihed*cos(theta-phi0)', '-k_dihed*cos_phicutoff'))
phiforce.addGlobalParameter('k_dihed', 100.0*kilojoules_per_mole)
phiforce.addGlobalParameter('phi0', (phi0*np.pi/180.0)*radians)
phiforce.addGlobalParameter('cos_phicutoff', cos(phicutoff*np.pi/180.0))

psiforce = CustomTorsionForce('select({},{},{})'.format('step(-k_dihed*cos(theta-psi0)-(-k_dihed*cos_psicutoff))', '-k_dihed*cos(theta-psi0)', '-k_dihed*cos_psicutoff'))
psiforce.addGlobalParameter('k_dihed', 100.0*kilojoules_per_mole)
psiforce.addGlobalParameter('psi0', (psi0*np.pi/180.0)*radians)
psiforce.addGlobalParameter('cos_psicutoff', cos(psicutoff*np.pi/180.0))


for i in range(len(phi_atoms_res_5_25)):
    phiforce.addTorsion(phi_atoms_res_5_25[i][0], phi_atoms_res_5_25[i][1], phi_atoms_res_5_25[i][2], phi_atoms_res_5_25[i][3])
    psiforce.addTorsion(psi_atoms_res_5_25[i][0], psi_atoms_res_5_25[i][1], psi_atoms_res_5_25[i][2], psi_atoms_res_5_25[i][3])
    
phiforce.setForceGroup(2)
psiforce.setForceGroup(3)
system.addForce(phiforce)
system.addForce(psiforce)


meta = Metadynamics(system, [dist_bias_1], 298.15*kelvin, 5.0, 1*kilojoule_per_mole, 1)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.004*picoseconds)

platform = Platform.getPlatformByName('CUDA')
properties = {'Precision': 'mixed'}

simulation = Simulation(prmtop.topology, system, integrator, platform, properties)

simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('test-output.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))

meta.step(simulation, 100000)

print ("simulation time:", time.time() - start_time, "s")


In [ ]:
traj = md.load('test-output.dcd', top = 'apart-lastframe.prmtop')

cv_tuple=[]

for crd in traj.xyz: #getting atom coordinates in each frame to make the simulation context
    simulation.context.setPositions(crd)
    cv_tuple.append(meta.getCollectiveVariables(simulation)) #get CV per frame

In [ ]:
fe = []
for i in meta.getFreeEnergy():
    fe.append(i._value)

In [ ]:
fe

In [ ]:
cv = []
for i in range(len(cv_tuple)):
    cv.append(cv_tuple[i][0])

In [ ]:
cv

In [ ]:
fig,ax = plt.subplots()
ax.plot(cv[1:], fe)
ax.set(xlabel='CV (nm)', ylabel='Free Energy (kJ/mol)', title='')
#ax.grid()
#ax.set_xlim([0.0, 20.0])

fig.savefig('fe-cv.png')
plt.show()

algebraic expression for calculating CV per frame - comparison with CV computed from simulation context

In [ ]:
g1_positions=[]
g2_positions=[]
g3_positions=[]
g4_positions=[]

#get x,y,z coordinates for each individual chain
for crd in traj.xyz:
    g1_positions.append(crd[g1[0]:g1[-1]+1]) 
    g2_positions.append(crd[g2[0]:g2[-1]+1])
    g3_positions.append(crd[g3[0]:g3[-1]+1])
    g4_positions.append(crd[g4[0]:g4[-1]+1])


cv_linalg = []

for idx in range(len(traj.xyz)):
    dist12 = np.linalg.norm(np.mean(np.asarray(g1_positions[idx]),axis=0) - np.mean(np.asarray(g2_positions[idx]), axis=0))
    dist13 = np.linalg.norm(np.mean(np.asarray(g1_positions[idx]),axis=0) - np.mean(np.asarray(g3_positions[idx]), axis=0))
    dist14 = np.linalg.norm(np.mean(np.asarray(g1_positions[idx]),axis=0) - np.mean(np.asarray(g4_positions[idx]), axis=0))
    dist23 = np.linalg.norm(np.mean(np.asarray(g2_positions[idx]),axis=0) - np.mean(np.asarray(g3_positions[idx]), axis=0))
    dist24 = np.linalg.norm(np.mean(np.asarray(g2_positions[idx]),axis=0) - np.mean(np.asarray(g4_positions[idx]), axis=0))
    dist34 = np.linalg.norm(np.mean(np.asarray(g3_positions[idx]),axis=0) - np.mean(np.asarray(g4_positions[idx]), axis=0))
    
    cv_lin = dist12+dist13+dist14+dist23+dist24+dist34
    cv_linalg.append(cv_lin)